### Functions for non-conserved species

In [1]:
def prep_lines(lines):
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines

'''
Read fortran ODEs and prep for translation.
    - file = file name
    - returns: dict
'''
def read_fortran_ODEs(file, cons = False):

    ## read fortran ODEs from given file
    with open(file, 'r') as f:
        lines = f.readlines()

    if cons == True:
        lines = lines[16:93]
    if cons == False:
        ## select non-conserved species
        lines = lines[123:]

    ## remove unnecessary fortran code
    lines = prep_lines(lines)

    ODEs = stick_fortran(lines)

    return ODEs

'''
make dict:
    - entry per spec
    - stick together separate fortran strings
'''
def stick_fortran(lines):
    
    ODEs = dict()

    for i, line in enumerate(lines):
        if len(line) != 0 and line[0] == 'C':
            ODEs[line[2:]] = str('')
            print('i=',i, end = '\r')
            start = i + 1

            for j in range(start,start+1000):
                if j >= len(lines):
                    break
                if len(lines[j]) != 0 and lines[j][0] == 'C':
                    stop = j
                    break
                
            for j in range(i+1,stop):
                ODEs[line[2:]] += lines[j]

            i = stop +1

    count = 0
    for key in ODEs:
        if key == '':
            count += 1

    while count > 0:
        del ODEs['']
        count -= 1

    return ODEs


'''
Translate fortran ODE to python ODE
'''
def fortran_to_python(ODEs, spec, cons = False):
    print(spec)

    if isinstance(ODEs[spec],str):
        ## NOT fractional abundances, but in units of cm^-3
        # ODEs[spec] = ODEs[spec].replace('*HNR', '')
        ## split string
        ODEs[spec] = ODEs[spec].split(' ')

    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## remove other fortran commands
    while 'END' in ODEs[spec]:
        ODEs[spec].remove('END')
    while 'RETURN' in ODEs[spec]:
        ODEs[spec].remove('RETURN')

    # print('Conserved? ',cons)
    ## Translate
    if cons == True:
        # print('Conserved? ',cons)
        ## replace brackets: ( --> [
        print(len(ODEs[spec]))
        for i in range(len(ODEs[spec])):
            
            # if ODEs[spec][i-1] != '+' or ODEs[spec][i-1] != '-' or ODEs[spec][i-1] != '*' or ODEs[spec][i-1] != '/' or ODEs[spec][i-1] != ')' or ODEs[spec][i-1] != '(':
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')
            ODEs[spec][i] = ODEs[spec][i].replace('[+', '(+')
            ODEs[spec][i] = ODEs[spec][i].replace(']]', '])')

    if cons == False:
        ## replace brackets: ( --> [
        for i in range(len(ODEs[spec])):
            if ODEs[spec][i][0] == 'F' or ODEs[spec][i][0] == 'D':
                ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
                ODEs[spec][i] = ODEs[spec][i].replace(')', ']')

        ## replace final line
        last = ODEs[spec][-1]
        # print('last', last)
        while not last[0] == 'Y':
            ODEs[spec].remove(last)
            last = ODEs[spec][-1]
        # print('at the end',last)
        for i in range(5,len(last)):
            if last[i] == ')':
                stop = i
                break
        nb = last[5:stop]

        if not spec == 'H':
            ODEs[spec][-1] = 'YDOT['+nb+']=F-(D*Y['+nb+'])'
        if spec == 'H':
            ODEs[spec][-1] = 'YDOT['+nb+']=(F+HLOSS)-(D*Y['+nb+'])'

            
    return ODEs
    

### Read in file & prep

In [2]:
file = '/lhome/silkem/CHEM/src-IP-AP-HNR/code/acodes.f'

with open(file, 'r') as f:
    lines = f.readlines()

ODEs = read_fortran_ODEs(file)


In [3]:
ODEs['C11+']

'      F=0.+K(1517)*Y(10)*Y(458)*HNR+K(1518)*Y(10)*Y(455)*HNR+K(2066)*Y(11)*Y(453)*HNR+K(2068)*Y(11)*Y(459)*HNR+K(5727)*Y(465)+K(6042)*Y(10)*Y(451)*HNR+K(6202)*Y(465)+K(6538)*Y(465)      D=0.+K(994)*X(1)*HNR+K(995)*X(1)*HNR+K(996)*X(1)*HNR+K(997)*X(1)*HNR      YDOT(466)=F-(D*Y(466))      RETURN      END'

### Translate

In [4]:
for key in ODEs:
    ODEs = fortran_to_python(ODEs, key)

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

### Conserved species

In [5]:
ODEs_cons = read_fortran_ODEs(file, True)

In [6]:
ODEs_cons

{'e-': '      X(1)=0.+TOTAL(1)+(+Y(2)-Y(3)+Y(4)+Y(5)+Y(7)+Y(8)-Y(9)+Y(10)-Y(12)+Y(13)+Y(15)+Y(16)+Y(21)+Y(22)+Y(23)+Y(24)+Y(25)-Y(26)-Y(30)+Y(32)+Y(33)+Y(34)+Y(36)+Y(38)+Y(40)+Y(41)+Y(43)+Y(44)+Y(46)-Y(47)+Y(48)+Y(50)-Y(52)+Y(53)+Y(57)-Y(58)+Y(59)+Y(62)+Y(63)+Y(66)+Y(67)+Y(69)+Y(70)+Y(73)+Y(75)+Y(76)+Y(78)+Y(79)+Y(83)+Y(84)+Y(85)+Y(87)+Y(90)+Y(91)+Y(93)+Y(94)+Y(95)+Y(97)+Y(98)+Y(99)+Y(101)+Y(102)+Y(108)+Y(110)+Y(111)-Y(112)-Y(113)+Y(114)+Y(115)+Y(116)+Y(117)+Y(119)+Y(121)+Y(123)+Y(124)+Y(126)+Y(128)+Y(129)+Y(131)+Y(132)-Y(134)+Y(136)+Y(137)-Y(138)+Y(139)+Y(142)+Y(143)+Y(145)+Y(147)+Y(148)+Y(150)+Y(153)+Y(155)+Y(156)+Y(159)+Y(160)+Y(162)+Y(163)+Y(164)+Y(166)+Y(168)+Y(171)+Y(172)+Y(175)+Y(177)+Y(179)+Y(181)+Y(182)+Y(184)+Y(187)+Y(188)+Y(189)+Y(192)+Y(193)+Y(195)+Y(196)+Y(197)+Y(198)+Y(200)+Y(201)+Y(202)+Y(203)+Y(204)+Y(206)+Y(209)+Y(210)+Y(212)+Y(213)+Y(215)+Y(216)+Y(217)+Y(218)+Y(221)+Y(222)+Y(223)+Y(224)+Y(229)+Y(230)+Y(233)+Y(235)+Y(236)+Y(237)+Y(239)+Y(240)+Y(241)+Y(242)+Y(243)+Y(245

In [7]:
for spec in ODEs_cons:
    ODEs_cons = fortran_to_python(ODEs_cons, spec, cons = True)

e-
1
H2
1


### Parse when line is too long


In [8]:
'''
Parse line for the conserved species
'''
def parse_line(parsed, cons = False):
    line = parsed[-1]

    
    if len(line) > 1000:
        if cons == False:
            for i in range(1000):
                n=i+1000
                if n < len(line):
                    if line[n] == '+' or line[n] == '-':
                        break
                else:
                    break
            parsed[-1] = line[0:n]
        
            parsed.append(line[0]+'='+line[0]+line[n:])            

    else: 
        print('no')
    
    return 


In [9]:
'''
Parse line for the non-conserved species
'''
def parse_line_cons():

        for i in range(1000):
            n=i+1000
            if n < len(part2):
                if part2[n] == '+' or part2[n] == '-':
                    break
            else:
                break


            
        parsed[-1] = line[0:n]
    
        parsed.append(line[0]+'='+line[0]+line[n:])

In [10]:
''' 
Parsing the ODEs of the conserved specs
'''
for spec in ODEs:
    print(spec)

    i = 0
    while i < len(ODEs[spec]):
        # print('i',i)

        if len(ODEs[spec][i]) > 1000:
            parsed = [ODEs[spec][i]]
            while len(parsed[-1]) > 1000:
                parse_line(parsed)

            # print('len parsed',len(parsed),'\n--\n')
            ODEs[spec] = ODEs[spec][0:i] + parsed + ODEs[spec][i+1:]

            i = i + len(parsed)
        
        else: i += 1

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

In [11]:
# for spec in ODEs_cons:
#     print(spec)    

#### WERKT NIET!!!!
spec = 'H2'
i = 0
while i < len(ODEs_cons[spec]):
    if len(ODEs_cons[spec][i]) > 1000:
        parsed = [ODEs_cons[spec][i]]
        print(parsed)

        for j in range(20):
            if ODEs_cons[spec][i][j] == '(' and not ODEs_cons[spec][i][j] == '*':
                print(j)
                break
        part1 = ODEs_cons[spec][i][0:j]
        parsed = ODEs_cons[spec][i][j+1:-1]

        print('\n----\n'+parsed)

        i = i + len(parsed)
    
    else: i += 1

['X[2]=0.+TOTAL[2]-0.5*(+Y[1]+Y[2]+Y[3]+2*Y[4]+3*Y[5]+Y[8]+Y[12]+Y[13]+Y[14]+2*Y[15]+2*Y[17]+3*Y[19]+Y[20]+Y[21]+3*Y[22]+2*Y[24]+4*Y[25]+2*Y[27]+4*Y[29]+Y[30]+Y[31]+Y[32]+5*Y[33]+3*Y[34]+3*Y[35]+4*Y[36]+2*Y[37]+2*Y[38]+3*Y[40]+Y[42]+Y[43]+2*Y[44]+Y[52]+Y[53]+Y[54]+2*Y[56]+2*Y[57]+Y[60]+3*Y[61]+3*Y[62]+Y[63]+Y[64]+4*Y[66]+2*Y[68]+2*Y[69]+4*Y[74]+2*Y[75]+5*Y[76]+Y[77]+Y[78]+Y[79]+3*Y[80]+Y[81]+5*Y[82]+Y[83]+Y[84]+4*Y[85]+2*Y[86]+6*Y[87]+2*Y[89]+4*Y[90]+2*Y[91]+6*Y[92]+2*Y[93]+3*Y[97]+Y[98]+3*Y[99]+Y[100]+7*Y[102]+3*Y[103]+4*Y[105]+Y[106]+2*Y[108]+4*Y[109]+4*Y[110]+Y[114]+4*Y[115]+5*Y[117]+Y[118]+2*Y[119]+Y[120]+Y[121]+2*Y[122]+Y[123]+5*Y[124]+2*Y[125]+2*Y[126]+2*Y[127]+3*Y[128]+3*Y[129]+Y[133]+Y[136]+2*Y[137]+Y[138]+Y[139]+Y[140]+2*Y[142]+2*Y[144]+2*Y[146]+Y[147]+3*Y[148]+3*Y[149]+3*Y[150]+4*Y[152]+4*Y[155]+2*Y[157]+4*Y[158]+2*Y[159]+5*Y[160]+Y[161]+Y[162]+3*Y[163]+Y[164]+3*Y[165]+6*Y[166]+4*Y[168]+2*Y[169]+6*Y[170]+2*Y[172]+2*Y[173]+2*Y[175]+2*Y[178]+Y[180]+Y[181]+Y[182]+Y[183]+3*Y[184]

In [12]:
string = 'X[2]=0.+TOTAL[2]-0.5*'
len(string)

21

### Write out in as python function

In [13]:
file = 'src/odes.py'

## Y = number density of unconserved species
## X = number density of conserved species
## K = reaction rate
## ACCR = loss of H due to dust interaction, kind of rate

with open(file, 'w') as f:
    f.write('import numpy as np\n')
    f.write("\n'''\nIMPORTANT NOTE:\nThe content of this function is translated from Fortran 77. \nHence, indexing starts from 1. At position 0 there is a dummy species.\n \nWritten by Silke Maes, April 2023.\nOriginal Fortran code by rate10odecsT.pl on Sun, Jul 21 2019, \nusing ratefile: rate16_IP_10000K.rates, from http://udfa.ajmarkwick.net/index.php. \n")
    f.write("\n------\n")
    f.write("\nY     = abundances of the non-conserved species")
    f.write("\nX     = abundances of the conserved species")
    f.write("\nK     = reaction rates, calculated in xxx")
    f.write("\nACCR  = H accretion onto dust, calculated in main.py from input physics")
    f.write("\nTOTAL = ??")
    f.write("\nHNR   = input density (ρ)")
    f.write("\n'''")
    f.write('\ndef ODE(Y, TOTAL, YDOT, X, K, HNR, ACCR):\n')
    f.write('\n\t## Conserved species\n')
    for key in ODEs_cons:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs_cons[key]:
            f.write('\t'+line+'\n')
    f.write('\n\n\t## Non-conserved species\n')
    f.write('\tHLOSS=-ACCR*Y[1]\n')
    for key in ODEs:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs[key]:
            f.write('\t'+line+'\n')
    f.write('\n\treturn X, YDOT, Y')

In [14]:
'''
ODEs for the non conserved species, manually parsed
'''


## e-
T = 0.+Y[2]-Y[3]+Y[4]+Y[5]+Y[7]+Y[8]-Y[9]+Y[10]-Y[12]+Y[13]+Y[15]+Y[16]+Y[21]+Y[22]+Y[23]+Y[24]+Y[25]-Y[26]-Y[30]+Y[32]+Y[33]+Y[34]+Y[36]+Y[38]+Y[40]+Y[41]+Y[43]+Y[44]+Y[46]-Y[47]+Y[48]+Y[50]-Y[52]+Y[53]+Y[57]-Y[58]+Y[59]+Y[62]+Y[63]+Y[66]+Y[67]+Y[69]+Y[70]+Y[73]+Y[75]+Y[76]+Y[78]+Y[79]+Y[83]+Y[84]+Y[85]+Y[87]+Y[90]+Y[91]+Y[93]+Y[94]+Y[95]+Y[97]+Y[98]+Y[99]+Y[101]+Y[102]+Y[108]+Y[110]+Y[111]-Y[112]-Y[113]+Y[114]+Y[115]+Y[116]+Y[117]+Y[119]+Y[121]+Y[123]+Y[124]+Y[126]+Y[128]+Y[129]+Y[131]+Y[132]-Y[134]+Y[136]+Y[137]-Y[138]+Y[139]+Y[142]+Y[143]+Y[145]+Y[147]+Y[148]+Y[150]+Y[153]+Y[155]+Y[156]+Y[159]+Y[160]+Y[162]+Y[163]+Y[164]+Y[166]+Y[168]+Y[171]+Y[172]+Y[175]+Y[177]+Y[179]+Y[181]+Y[182]+Y[184]+Y[187]+Y[188]+Y[189]+Y[192]+Y[193]+Y[195]+Y[196]+Y[197]+Y[198]+Y[200]+Y[201]+Y[202]+Y[203]+Y[204]+Y[206]+Y[209]+Y[210]+Y[212]+Y[213]+Y[215]+Y[216]+Y[217]+Y[218]+Y[221]+Y[222]+Y[223]+Y[224]+Y[229]+Y[230]+Y[233]+Y[235]+Y[236]+Y[237]+Y[239]+Y[240]+Y[241]+Y[242]+Y[243]+Y[245]+Y[246]+Y[247]+Y[248]+Y[249]+Y[250]+Y[252]+Y[253]-Y[254]
T = T+Y[255]+Y[259]+Y[260]+Y[261]+Y[262]+Y[263]+Y[264]-Y[266]+Y[267]+Y[268]+Y[271]-Y[272]+Y[273]+Y[277]+Y[278]+Y[283]+Y[285]+Y[286]+Y[287]+Y[288]+Y[289]+Y[290]+Y[293]+Y[294]+Y[295]+Y[296]+Y[297]+Y[299]+Y[301]+Y[303]+Y[306]+Y[307]+Y[308]+Y[309]+Y[311]+Y[312]+Y[313]+Y[315]+Y[317]+Y[318]+Y[319]+Y[321]+Y[322]+Y[323]+Y[324]+Y[327]-Y[329]+Y[330]+Y[333]+Y[334]+Y[335]+Y[336]+Y[337]+Y[338]+Y[340]-Y[341]+Y[342]+Y[343]+Y[346]+Y[347]+Y[351]+Y[352]+Y[353]+Y[355]+Y[356]+Y[358]+Y[359]+Y[360]+Y[361]+Y[364]+Y[365]+Y[366]+Y[367]+Y[369]+Y[370]+Y[372]+Y[374]+Y[375]-Y[376]+Y[377]+Y[380]-Y[381]+Y[383]-Y[385]+Y[386]+Y[387]+Y[389]+Y[390]+Y[391]+Y[392]+Y[394]+Y[395]+Y[396]+Y[398]+Y[400]+Y[401]+Y[403]+Y[404]+Y[405]+Y[406]-Y[408]-Y[409]+Y[410]+Y[412]+Y[414]+Y[415]+Y[418]+Y[419]-Y[420]+Y[422]+Y[424]-Y[425]+Y[428]+Y[429]+Y[431]+Y[432]+Y[433]+Y[434]+Y[435]+Y[436]+Y[438]-Y[439]+Y[440]-Y[441]+Y[443]+Y[445]+Y[446]+Y[448]+Y[449]-Y[450]+Y[452]+Y[453]-Y[454]+Y[457]+Y[459]+Y[460]+Y[461]+Y[463]+Y[464]+Y[466]
X[1]=0.+TOTAL[1]+T

## H2
T = 0.+Y[1]+Y[2]+Y[3]+2*Y[4]+3*Y[5]+Y[8]+Y[12]+Y[13]+Y[14]+2*Y[15]+2*Y[17]+3*Y[19]+Y[20]+Y[21]+3*Y[22]+2*Y[24]+4*Y[25]+2*Y[27]+4*Y[29]+Y[30]+Y[31]+Y[32]+5*Y[33]+3*Y[34]+3*Y[35]+4*Y[36]+2*Y[37]+2*Y[38]+3*Y[40]+Y[42]+Y[43]+2*Y[44]+Y[52]+Y[53]+Y[54]+2*Y[56]+2*Y[57]+Y[60]+3*Y[61]+3*Y[62]+Y[63]+Y[64]+4*Y[66]+2*Y[68]+2*Y[69]+4*Y[74]+2*Y[75]+5*Y[76]+Y[77]+Y[78]+Y[79]+3*Y[80]+Y[81]+5*Y[82]+Y[83]+Y[84]+4*Y[85]+2*Y[86]+6*Y[87]+2*Y[89]+4*Y[90]+2*Y[91]+6*Y[92]+2*Y[93]+3*Y[97]+Y[98]+3*Y[99]+Y[100]+7*Y[102]+3*Y[103]+4*Y[105]+Y[106]+2*Y[108]+4*Y[109]+4*Y[110]+Y[114]+4*Y[115]+5*Y[117]+Y[118]+2*Y[119]+Y[120]+Y[121]+2*Y[122]+Y[123]+5*Y[124]+2*Y[125]+2*Y[126]+2*Y[127]+3*Y[128]+3*Y[129]+Y[133]+Y[136]+2*Y[137]+Y[138]+Y[139]+Y[140]+2*Y[142]+2*Y[144]+2*Y[146]+Y[147]+3*Y[148]+3*Y[149]+3*Y[150]+4*Y[152]+4*Y[155]+2*Y[157]+4*Y[158]+2*Y[159]+5*Y[160]+Y[161]+Y[162]+3*Y[163]+Y[164]+3*Y[165]+6*Y[166]+4*Y[168]+2*Y[169]+6*Y[170]+2*Y[172]+2*Y[173]+2*Y[175]+2*Y[178]+Y[180]+Y[181]+Y[182]+Y[183]+3*Y[184]+Y[185]+7*Y[187]+3*Y[188]+Y[189]+Y[190]+Y[191]
T = T+3*Y[192]+Y[193]+3*Y[194]+Y[195]+2*Y[197]+4*Y[200]+2*Y[202]+2*Y[203]+2*Y[204]+Y[205]+4*Y[207]+2*Y[209]+4*Y[210]+2*Y[212]+Y[213]+5*Y[215]+Y[216]+Y[217]+Y[220]+2*Y[221]+Y[222]+Y[223]+2*Y[225]+2*Y[226]+3*Y[228]+2*Y[229]+6*Y[230]+6*Y[232]+2*Y[233]+2*Y[234]+3*Y[235]+2*Y[236]+6*Y[237]+6*Y[238]+Y[240]+7*Y[241]+2*Y[243]+3*Y[245]+Y[248]+4*Y[249]+3*Y[250]+7*Y[252]+3*Y[253]+Y[255]+Y[256]+3*Y[261]+Y[262]+2*Y[263]+2*Y[264]+Y[265]+Y[266]+Y[267]+2*Y[268]+2*Y[270]+Y[273]+3*Y[274]+Y[275]+3*Y[278]+Y[279]+4*Y[281]+2*Y[285]+4*Y[286]+Y[289]+5*Y[290]+Y[291]+3*Y[292]+3*Y[293]+Y[294]+Y[295]+2*Y[296]+4*Y[297]+6*Y[298]+2*Y[299]+2*Y[300]+Y[303]+5*Y[305]+3*Y[306]+7*Y[308]+3*Y[309]+Y[310]+Y[311]+6*Y[313]+Y[317]+2*Y[318]+6*Y[319]+6*Y[320]+3*Y[321]+7*Y[322]+3*Y[323]+4*Y[324]+4*Y[325]+4*Y[334]+Y[335]+Y[336]+Y[337]+Y[338]+Y[339]+5*Y[340]+Y[341]+2*Y[342]+2*Y[344]+Y[346]+3*Y[347]+4*Y[348]+4*Y[352]+2*Y[355]+Y[357]+Y[358]+Y[359]+3*Y[360]+Y[361]+3*Y[362]+Y[363]+5*Y[364]+4*Y[365]+2*Y[366]+2*Y[367]+2*Y[368]+3*Y[369]+3*Y[370]+Y[372]+Y[375]+Y[379]
T = T+Y[380]+Y[381]+2*Y[382]+2*Y[383]+Y[387]+Y[388]+3*Y[389]+4*Y[391]+2*Y[392]+3*Y[394]+Y[395]+5*Y[396]+6*Y[397]+6*Y[398]+7*Y[400]+Y[403]+Y[405]+Y[409]+Y[410]+Y[411]+2*Y[412]+2*Y[413]+3*Y[414]+4*Y[415]+4*Y[416]+3*Y[417]+5*Y[418]+4*Y[419]+Y[423]+Y[424]+Y[425]+2*Y[426]+2*Y[429]+Y[430]+Y[431]+3*Y[432]+4*Y[433]+2*Y[434]+5*Y[435]+3*Y[436]+Y[440]+Y[441]+Y[442]+2*Y[443]+2*Y[444]+3*Y[445]+4*Y[446]+3*Y[447]+5*Y[448]+4*Y[449]+Y[453]+Y[454]+Y[455]+2*Y[458]+2*Y[459]+Y[460]+3*Y[461]+Y[462]+2*Y[463]+3*Y[464]
X[2]=0.+TOTAL[2]-0.5*T


NameError: name 'Y' is not defined